In [1]:
# yolov8 seg모델 다운
# https://docs.ultralytics.com/ko/tasks/segment/#models

from ultralytics import YOLO

model = YOLO('yolov8m-seg.pt')  

100%|██████████| 52.4M/52.4M [00:04<00:00, 12.9MB/s]


In [3]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from ultralytics import YOLO
import numpy as np
import json

# 목표 함수 정의
def objective(params):
    model = YOLO('yolov8m-seg.pt')
    
    # 모델 훈련
    model.train(data="/data.yaml", 
                epochs=10,  # 초기 탐색을 위한 적당한 에폭 수 설정
                batch=16, 
                lr0=params['lr0'], 
                lrf=params['lrf'], 
                momentum=params['momentum'], 
                weight_decay=params['weight_decay'], 
                optimizer=params['optimizer']
               )
    
    # 모델 평가
    results = model.val(data="data.yaml")
    
    # 원하는 평가 지표 선택 (예: mAP@50)
    map50 = results.box.map50
    
    return {'loss': -map50, 'status': STATUS_OK}

# 검색 공간 설정
space = {
    'lr0': hp.loguniform('lr0', np.log(0.001), np.log(0.02)),  # 기존 최적 값 0.01을 중심으로 범위 설정
    'lrf': hp.uniform('lrf', 0.005, 0.015),  # 기존 최적 값 0.01을 중심으로 범위 설정
    'momentum': hp.uniform('momentum', 0.9, 0.99),  # 기존 최적 값 0.937을 중심으로 범위 설정
    'weight_decay': hp.loguniform('weight_decay', np.log(0.0001), np.log(0.001)),  # 기존 최적 값 0.0005을 중심으로 범위 설정
    'optimizer': hp.choice('optimizer', ['SGD', 'Adam'])
}

# 최적화 수행
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# 최적 하이퍼파라미터 출력
print("Best hyperparameters found:")
print(best)

# 최적 하이퍼파라미터를 JSON 파일로 저장
output_path = "best_hyperparameters.json"
with open(output_path, 'w') as f:
    json.dump(best, f, indent=4)

print(f"Best hyperparameters have been saved to {output_path}")


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]New https://pypi.org/project/ultralytics/8.2.48 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.31  Python-3.11.8 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=C:/Users/SSTLabs/Desktop/test2/SSTLabs-2-5/data.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, aug

train: Scanning C:\Users\SSTLabs\Desktop\test2\SSTLabs-2-5\train\labels.cache... 7285 images, 3 backgrounds, 0 corrupt: 100%|##########| 7285/7285 [00:00<?, ?it/s]
train: Scanning C:\Users\SSTLabs\Desktop\test2\SSTLabs-2-5\train\labels.cache... 7285 images, 3 backgrounds, 0 corrupt: 100%|##########| 7285/7285 [00:00<?, ?it/s]
val: Scanning C:\Users\SSTLabs\Desktop\test2\SSTLabs-2-5\valid\labels.cache... 734 images, 0 backgrounds, 0 corrupt: 100%|##########| 734/734 [00:00<?, ?it/s]
val: Scanning C:\Users\SSTLabs\Desktop\test2\SSTLabs-2-5\valid\labels.cache... 734 images, 0 backgrounds, 0 corrupt: 100%|##########| 734/734 [00:00<?, ?it/s]


Plotting labels to runs\segment\train3\labels.jpg... 
optimizer: SGD(lr=0.006116958284391689, momentum=0.921283230661808) with parameter groups 86 weight(decay=0.0), 97 weight(decay=0.0006425365859054679), 96 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train3
Starting training for 10 epochs...
Closing dataloader mosaic
